# 使用marker + langchain 完整rag流程

In [3]:
from marker.converters.pdf import PdfConverter #将pdf内容转换为中间输出结果 rendered 
from marker.models import create_model_dict # 初始化并返回模型字典，字典中包括ocr、版面分析、文本识别等要使用的模型
from marker.output import text_from_rendered #从pdfconverter生成的rendered结构化结果中，提取text、分页信息、图像等，并输出为
from marker.config.parser import ConfigParser

config = {
    "output_format": "json",    
}
config_parser = ConfigParser(config)

converter = PdfConverter(
    config = config_parser.generate_config_dict(),
    renderer = config_parser.get_renderer(),
    artifact_dict = create_model_dict()
)
rendered = converter(r"knowledgeBase\yingzao_chapter01.pdf") #将 converter 当作函数调用（说明其实现了 __call__ 方法）
text, _, images = text_from_rendered(rendered) # （md文本、md格式、图片集）


Recognizing Text: 100%|██████████| 276/276 [09:36<00:00,  2.09s/it] 
Detecting bboxes: 0it [00:00, ?it/s]


In [12]:
print(type(text))
print(len(text))
print(text[0:5000])

<class 'str'>
20452


{0}------------------------------------------------

# 第一章 总 论

#### 一、《营造法式》的性质与特点

#### (一)"营造法式"是一种建筑工程预算定额

"法式"二字在宋代用得相当普遍,有律令、条例、定式等含义,凡是有明文规定或成法的都可称之为法式,从以下数例可以得到说明:

其一,《宋会要辑稿》第五十八册,职官一:"熙宁五年(1072年)二月一日诏:'……以事因贴奏,诸称奏者,有法式上门下省,无法式上中书省;有别条者依本法'";

其二,《宋史·职官志》:"门下省,……尚书省、六部所上有法式事,皆奏覆审驳之";"中书省,掌进拟庶务、宣奉命令……及中外无法式事应取旨事,……凡事干因革损益而非法式所裁者,论定而上之"。

可见"法式"二字所包含的内容是相当广泛的,它的性质是政府所制定的法令与成规。由此推论,《营造法式》应是政府对建筑工程所制定的在实际工作中必须遵照执行的法规,而不是没有约束力的技术性著作。再从李诫《营造法式》的序和劄子来看,宋朝政府要求的实际上就是一种建筑工程预算定额,用它来节制各项工程的财政开支。梁思成先生的《营造法式注释》序中说:"《营造法式》是北宋官订的建筑设计、施工的专书。它的性质略似于今天的设计手册加上建筑规范。"这个定性虽然也有道理,但从根本上说,两者还是有区别的。为了说明这一点,首先来看看营造法式产生的历史背景。

北宋时,特别是北宋中晚期,建筑业弊端丛生,腐败日甚。一些负责修缮京师房屋的官员在工程开始之前往往多估工料,虚报项目;施工过程中偷工减料,监守自盗;工程结束后又谎报结余,邀功请赏。工程质量也因此大受影响,房屋多不坚固。为此,早在宋仁宗天圣元年(1023年)三司(总管国家经济财政,位次宰相,下设盐铁、度支、户部三使)就提出要加强工程主管官员和都料(设计施工负责人)、作头(各工种的工头)的责任制,严格工程项目的立项以及工料估算的检查,并要求各司"……将见修三、五间舍屋,以所破功料,须委监修,相度日用功力,计定功限,永为定式"(1)。这里所说的"计定功限,永为定式"已具有估算定额的性质,是营造法式的雏形。不过三司提出的办法似乎并未很好执行,建筑业的积弊也远未消除,所以到仁宗至和元年(1054年)又下诏:"比闻差官修缮京师官

In [13]:
print(len(_))
print(_[0:200])

2
md


## 保存pdf解析结果到本地

In [14]:
import os
from datetime import datetime

output_dir = r"knowledgeBase\output"
os.makedirs(output_dir, exist_ok=True)

# 2. 生成一个时间戳，避免覆盖旧结果
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# 3. 保存文本 / JSON 结果
text_output_path = os.path.join(output_dir, f"result_{timestamp}.json")
with open(text_output_path, "w", encoding="utf-8") as f:
    f.write(text)

# 4. 保存图片（如果有）
image_output_dir = os.path.join(output_dir, f"images_{timestamp}")
saved_images = []

if images:
    os.makedirs(image_output_dir, exist_ok=True)
    for idx, (image_name, image_data) in enumerate(images.items()):
        # image_data 通常是 bytes 或 PIL Image，不同 renderer 行为略有差异
        image_path = os.path.join(image_output_dir, image_name)

        if hasattr(image_data, "save"):  # PIL Image
            image_data.save(image_path)
        else:  # bytes
            with open(image_path, "wb") as img_f:
                img_f.write(image_data)

        saved_images.append(image_path)

# 5. Notebook 中输出结果说明
print("✅ PDF 解析结果已保存")
print(f"📄 文本/JSON 文件: {text_output_path}")

if saved_images:
    print(f"🖼 图片目录: {image_output_dir}")
    print(f"   图片数量: {len(saved_images)}")
else:
    print("🖼 本次解析未产生可导出的图片")


✅ PDF 解析结果已保存
📄 文本/JSON 文件: knowledgeBase\output\result_20260107_153320.json
🖼 图片目录: knowledgeBase\output\images_20260107_153320
   图片数量: 31


# mock chunks test totall pipline

In [4]:
import json
from langchain_core.documents import Document

chunks_file_path = r"knowledgeBase\chunks\my_mock_chunks.jsonl"

langch_docs = []
with open(chunks_file_path, "r", encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        langch_docs.append(
            Document(
                page_content = obj["chunk_content"],
                metadata = obj["metadata"]
            )
        )


In [4]:
import pprint

print((len(langch_docs)))
pprint.pprint(langch_docs[10].page_content)
print(pprint.pformat(langch_docs[10].metadata))
print(langch_docs[10].metadata.get("chunk_id"))

21
'《营造法式》在叙述“制度”做法时，只讲其然，不讲其所以然，对这些做法的功能原因、结构原因及审美原因一概不予涉及。例如，阑干部分仅收录了重台钩阑和单钩阑两种，而忽略了其他常见的阑干式样。这种叙述方式反映了《法式》作为建筑工程预算定额的性质，而非全面的建筑设计施工手册。'
{'Item': None,
 'Item_index': 1,
 'authors': ['潘谷西', '何建中'],
 'book_edition_time': '2017-03-01修订版',
 'book_title': '《营造法式》解读',
 'chapter': '第一章 总论',
 'chapter_index': 1,
 'chunk_content_type': 'main_text',
 'chunk_id': 'yingzao_2017_ch1_sec3_0002',
 'has_image': False,
 'images': [{'caption': None, 'figure_id': None, 'page': None, 'path': None}],
 'page_end': 14,
 'page_start': 13,
 'section': '三、《营造法式》的内容取舍',
 'section_index': 3,
 'subsection': None,
 'toc_path': ['第一章 总论', '三、《营造法式》的内容取舍']}
yingzao_2017_ch1_sec3_0002


In [5]:
from langchain_community.embeddings import OllamaEmbeddings

embedding_model = OllamaEmbeddings(
    model = "nomic-embed-text:latest",
    base_url = "http://localhost:11434"
)
print("Ollama embedding model loaded.")

# 从langch_docs提取文本内容进行嵌入计算
texts = [doc.page_content for doc in langch_docs]
print((len(texts)))
embeddings = embedding_model.embed_documents(texts)

# 将嵌入与对应的chunk_id和metadata关联起来
embedded_chunks = []
for doc, vector in zip(langch_docs, embeddings):
    embedded_chunks.append({
        "chunk_id": doc.metadata.get("chunk_id"),
        "embedding": vector,
        "metadata": doc.metadata
    })


Ollama embedding model loaded.
21


In [21]:
from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_documents(
    documents = langch_docs,
    embedding = embedding_model
)
print("FAISS vectorstore created.")
print(f"Total vectors stored: {vectorstore.index.ntotal}")




FAISS vectorstore created.
Total vectors stored: 21


In [24]:
# 创建一个检索器，设置返回最相似的 chunk 数量
complex_filter = {
    "$and": [
        {"chapter_index": {"$eq": 1}},
        {"has_image": {"$eq": True}}
    ]
}

In [26]:
# ===== 9. 测试查询 =====
retriever = vectorstore.as_retriever(
    search_kwargs={
        "k": 3,
        "filter": complex_filter
    }
)
query = "《营造法式》与江南建筑的关系"
results = retriever.invoke(query)

print(f"Query: {query}")
print(f"Returned chunks: {len(results)}")
# ===== 10. 打印返回的 chunk 内容与元数据 =====
for i, doc in enumerate(results, start=1):
    print("=" * 80)
    print(f"[Result {i}]")
    print("Chunk ID:", doc.metadata.get("chunk_id"))
    print("Chapter:", doc.metadata.get("chapter"))
    print("Section:", doc.metadata.get("section"))
    print("Image:", doc.metadata.get("has_image"))
    print("Subsection:", doc.metadata.get("subsection"))
    print("Pages:", doc.metadata.get("page_start"), "-", doc.metadata.get("page_end"))
    print("-" * 40)
    print(doc.page_content[:200])  # 只打印前 500 字，防止刷屏


Query: 《营造法式》与江南建筑的关系
Returned chunks: 3
[Result 1]
Chunk ID: yingzao_2017_ch1_sum_0002
Chapter: 第一章 总论
Section: None
Image: True
Subsection: None
Pages: 4 - 12
----------------------------------------
《营造法式》与江南建筑有着密切的关系，其记载的拼柱法、竹材应用、串构件、蒜瓣柱与梭柱、上昂等做法，在江南地区宋代建筑中屡见不鲜。这些做法不仅美观，而且具有良好的力学性能，反映了江南建筑对自然材料的巧妙利用和技术创新。同时，江南与汴京之间的技术交流也促进了《法式》的编纂和推广。
[Result 2]
Chunk ID: yingzao_2017_ch1_sec2_0004
Chapter: 第一章 总论
Section: 二、《营造法式》与江南建筑的关系
Image: True
Subsection: None
Pages: 7 - 8
----------------------------------------
《营造法式》中记载的蒜瓣柱与梭柱，在江南地区宋代建筑中屡见不鲜，如苏州虎丘云岩寺塔、罗汉院大殿等。这些柱式不仅美观，而且具有良好的力学性能。然而，在北方地区却较为罕见，反映了南北建筑风格的差异。此外，上昂构件在江南地区的应用也极为广泛，且早于《法式》的记载，显示了江南建筑在技术创新方面的领先地位。
[Result 3]
Chunk ID: yingzao_2017_ch1_sec2_0001
Chapter: 第一章 总论
Section: 二、《营造法式》与江南建筑的关系
Image: True
Subsection: None
Pages: 4 - 5
----------------------------------------
浙江宁波保国寺大殿的拼柱法，是北宋时期拼合柱的孤例，与《营造法式》中记载的拼柱法不谋而合，反映了当时解决大料困难的智慧。此外，江南地区宋代建筑中的竹材使用、串构件、蒜瓣柱与梭柱、上昂等做法，与《营造法式》中的记载相印证，显示了江南建筑对《法式》的深远影响。这些做法在北方较为少见，进一步证明了《法式》与江南建筑的紧密联系。


In [27]:
# ===== 12. 查看相似度得分（调 chunk / embedding 用） =====
docs_with_scores = vectorstore.similarity_search_with_score(query, k=5)

for doc, score in docs_with_scores:
    print("=" * 60)
    print("Score:", score)
    print("Chunk ID:", doc.metadata.get("chunk_id"))
    print(doc.page_content[:300])


Score: 234.37265
Chunk ID: yingzao_2017_ch1_sum_0002
《营造法式》与江南建筑有着密切的关系，其记载的拼柱法、竹材应用、串构件、蒜瓣柱与梭柱、上昂等做法，在江南地区宋代建筑中屡见不鲜。这些做法不仅美观，而且具有良好的力学性能，反映了江南建筑对自然材料的巧妙利用和技术创新。同时，江南与汴京之间的技术交流也促进了《法式》的编纂和推广。
Score: 283.88727
Chunk ID: yingzao_2017_ch1_sec2_0004
《营造法式》中记载的蒜瓣柱与梭柱，在江南地区宋代建筑中屡见不鲜，如苏州虎丘云岩寺塔、罗汉院大殿等。这些柱式不仅美观，而且具有良好的力学性能。然而，在北方地区却较为罕见，反映了南北建筑风格的差异。此外，上昂构件在江南地区的应用也极为广泛，且早于《法式》的记载，显示了江南建筑在技术创新方面的领先地位。
Score: 286.09085
Chunk ID: yingzao_2017_ch1_sec2_0001
浙江宁波保国寺大殿的拼柱法，是北宋时期拼合柱的孤例，与《营造法式》中记载的拼柱法不谋而合，反映了当时解决大料困难的智慧。此外，江南地区宋代建筑中的竹材使用、串构件、蒜瓣柱与梭柱、上昂等做法，与《营造法式》中的记载相印证，显示了江南建筑对《法式》的深远影响。这些做法在北方较为少见，进一步证明了《法式》与江南建筑的紧密联系。
Score: 294.7055
Chunk ID: yingzao_2017_ch1_sec2_0005
《营造法式》中的“七朱八白”装饰，在江南五代至北宋间的建筑物上广泛应用，如杭州灵隐寺石塔、苏州虎丘塔等。这种装饰不仅美观，而且具有一定的文化意义。然而，在北方地区，虽然也有类似图案的出现，但在辽、金建筑上却未见此式，反映了南北建筑装饰风格的差异。此外，《法式》中的令栱长度规定也与江南建筑实践相符，进一步证明了《法式》与江南建筑的紧密联系。
Score: 358.60773
Chunk ID: yingzao_2017_ch1_sec2_0002
《营造法式》中竹材的广泛应用，不仅体现在殿阁、厅堂等高级建筑中，也深入到宫廷生活的各个方面。竹笆、竹篾编成的各种构件，如心柱编竹造、隔截编道、篾辫索等，展现了竹材在建筑中的多功能性。此外，竹材还用于壁画加固、临时凉棚等，

## 链接本地LLM，构造回答

In [28]:
from langchain_community.llms import Ollama

llm = Ollama(
    model="gemma3:latest", 
    base_url="http://localhost:11434",
    temperature=0.2
)
print("LLM loaded.")

e:\Miniconda\envs\rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


LLM loaded.


C:\Users\孙俊强\AppData\Local\Temp\ipykernel_23004\3231409661.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  llm = Ollama(


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
你是一名严谨的中国传统木建筑学术助理，请仅根据【参考资料】回答问题。

要求：
1. 回答应准确、学术化，使用专业术语
2. 不要编造资料中不存在的内容，如果资料中没有提及，则说明“不在参考资料中”
3. 回答末尾以【引用】形式列出使用到的 chunk_id toc_path和页码

【参考资料】
{context}

【问题】
{question}

【回答】
"""
)

In [46]:
def format_docs_with_citation(docs):
    formatted = []

    for doc in docs:
        metadata = doc.metadata or {}

        chunk_id = metadata.get("chunk_id", "unknown")
        page_start = metadata.get("page_start", "?")
        page_end = metadata.get("page_end", "?")
        toc_path = metadata.get("toc_path")

        if isinstance(toc_path, (list, tuple)):
            toc_path_str = ",".join(toc_path)
        elif isinstance(toc_path, str):
            toc_path_str = toc_path
        else:
            toc_path_str = "unknown"

        formatted.append(
            f"[chunk_id: {chunk_id}, toc_path: {toc_path_str}, pages: {page_start}-{page_end}]\n"
            f"{doc.page_content}"
        )

    return "\n\n".join(formatted)


In [43]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {
        "context": retriever | format_docs_with_citation,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
)


In [54]:
query = "江南建筑和营造法式的关系"

answer = rag_chain.invoke(query)

print(answer)


《营造法式》与江南建筑有着密切的关系，其记载的拼柱法、竹材应用、串构件、蒜瓣柱与梭柱、上昂等做法，在江南地区宋代建筑中屡见不鲜。同时，江南与汴京之间的技术交流促进了《法式》的编纂和推广。在具体建筑中，如宁波保国寺的拼柱法、苏州虎丘云岩寺塔和罗汉院大殿中的蒜瓣柱与梭柱，以及江南地区广泛应用的上昂构件，都与《营造法式》的记载相印证，反映了江南建筑对《法式》的深远影响。【引用：yingzao_2017_ch1_sum_0002, 4-12; yingzao_2017_ch1_sec2_0001, 4-5; yingzao_2017_ch1_sec2_0004, 7-8】



### 以下为显示流程

In [52]:
query = "《营造法式》和江南建筑的关系是什么？"

retrieved_docs = retriever.invoke(query)

print(f"Retrieved {len(retrieved_docs)} chunks.\n")

for i, doc in enumerate(retrieved_docs, start=1):
    print("=" * 80)
    print(f"[Chunk {i}]")
    print("chunk_id:", doc.metadata.get("chunk_id"))
    print("chapter:", doc.metadata.get("chapter"))
    print("pages:", doc.metadata.get("page_start"), "-", doc.metadata.get("page_end"))
    print("-" * 40)
    print(doc.page_content[:500]) 

Retrieved 3 chunks.

[Chunk 1]
chunk_id: yingzao_2017_ch1_sum_0002
chapter: 第一章 总论
pages: 4 - 12
----------------------------------------
《营造法式》与江南建筑有着密切的关系，其记载的拼柱法、竹材应用、串构件、蒜瓣柱与梭柱、上昂等做法，在江南地区宋代建筑中屡见不鲜。这些做法不仅美观，而且具有良好的力学性能，反映了江南建筑对自然材料的巧妙利用和技术创新。同时，江南与汴京之间的技术交流也促进了《法式》的编纂和推广。
[Chunk 2]
chunk_id: yingzao_2017_ch1_sec2_0001
chapter: 第一章 总论
pages: 4 - 5
----------------------------------------
浙江宁波保国寺大殿的拼柱法，是北宋时期拼合柱的孤例，与《营造法式》中记载的拼柱法不谋而合，反映了当时解决大料困难的智慧。此外，江南地区宋代建筑中的竹材使用、串构件、蒜瓣柱与梭柱、上昂等做法，与《营造法式》中的记载相印证，显示了江南建筑对《法式》的深远影响。这些做法在北方较为少见，进一步证明了《法式》与江南建筑的紧密联系。
[Chunk 3]
chunk_id: yingzao_2017_ch1_sec2_0004
chapter: 第一章 总论
pages: 7 - 8
----------------------------------------
《营造法式》中记载的蒜瓣柱与梭柱，在江南地区宋代建筑中屡见不鲜，如苏州虎丘云岩寺塔、罗汉院大殿等。这些柱式不仅美观，而且具有良好的力学性能。然而，在北方地区却较为罕见，反映了南北建筑风格的差异。此外，上昂构件在江南地区的应用也极为广泛，且早于《法式》的记载，显示了江南建筑在技术创新方面的领先地位。


In [53]:
# ===== 2. 构造 context（与 RAG Chain 中完全一致）=====
context_text = format_docs_with_citation(retrieved_docs)
print(len(context_text))
print("=" * 80)
print("Context passed to LLM:\n")
print(context_text)  # 防止过长


693
Context passed to LLM:

[chunk_id: yingzao_2017_ch1_sum_0002, toc_path: 第一章 总论, pages: 4-12]
《营造法式》与江南建筑有着密切的关系，其记载的拼柱法、竹材应用、串构件、蒜瓣柱与梭柱、上昂等做法，在江南地区宋代建筑中屡见不鲜。这些做法不仅美观，而且具有良好的力学性能，反映了江南建筑对自然材料的巧妙利用和技术创新。同时，江南与汴京之间的技术交流也促进了《法式》的编纂和推广。

[chunk_id: yingzao_2017_ch1_sec2_0001, toc_path: 第一章 总论,二、《营造法式》与江南建筑的关系, pages: 4-5]
浙江宁波保国寺大殿的拼柱法，是北宋时期拼合柱的孤例，与《营造法式》中记载的拼柱法不谋而合，反映了当时解决大料困难的智慧。此外，江南地区宋代建筑中的竹材使用、串构件、蒜瓣柱与梭柱、上昂等做法，与《营造法式》中的记载相印证，显示了江南建筑对《法式》的深远影响。这些做法在北方较为少见，进一步证明了《法式》与江南建筑的紧密联系。

[chunk_id: yingzao_2017_ch1_sec2_0004, toc_path: 第一章 总论,二、《营造法式》与江南建筑的关系, pages: 7-8]
《营造法式》中记载的蒜瓣柱与梭柱，在江南地区宋代建筑中屡见不鲜，如苏州虎丘云岩寺塔、罗汉院大殿等。这些柱式不仅美观，而且具有良好的力学性能。然而，在北方地区却较为罕见，反映了南北建筑风格的差异。此外，上昂构件在江南地区的应用也极为广泛，且早于《法式》的记载，显示了江南建筑在技术创新方面的领先地位。


In [50]:
# ===== 3. 显式调用 LLM =====
final_prompt = prompt.invoke({
    "context": context_text,
    "question": query
})

print("=" * 80)
print("Prompt sent to LLM:\n")
print(final_prompt)


Prompt sent to LLM:

messages=[HumanMessage(content='\n你是一名严谨的中国传统木建筑学术助理，请仅根据【参考资料】回答问题。\n\n要求：\n1. 回答应准确、简明、学术化，使用专业术语\n2. 不要编造资料中不存在的内容，如果资料中没有提及，则说明“不在参考资料中”\n3. 回答末尾以【引用】形式列出使用到的 chunk_id toc_path和页码\n\n【参考资料】\n[chunk_id: yingzao_2017_ch1_sec3_0003, toc_path: 第一章 总论,三、《营造法式》的内容取舍, pages: 14-15]\n在石作雕镌制度方面，《营造法式》详细记载了四种雕法，但在实际建筑中，雕刻品类更为丰富。例如，“实雕”和“平钑”两种雕刻方式，在实物中屡见不鲜，却未被《法式》收录。这可能是因为《法式》作为预算定额，更注重用料的规范性和经济性，而非全面记录雕刻艺术的多样性。这些遗漏为后世研究宋代建筑雕刻艺术提供了新的课题。\n\n[chunk_id: yingzao_2017_ch1_sum_0002, toc_path: 第一章 总论, pages: 4-12]\n《营造法式》与江南建筑有着密切的关系，其记载的拼柱法、竹材应用、串构件、蒜瓣柱与梭柱、上昂等做法，在江南地区宋代建筑中屡见不鲜。这些做法不仅美观，而且具有良好的力学性能，反映了江南建筑对自然材料的巧妙利用和技术创新。同时，江南与汴京之间的技术交流也促进了《法式》的编纂和推广。\n\n[chunk_id: yingzao_2017_ch1_sec2_0005, toc_path: 第一章 总论,二、《营造法式》与江南建筑的关系, pages: 10-11]\n《营造法式》中的“七朱八白”装饰，在江南五代至北宋间的建筑物上广泛应用，如杭州灵隐寺石塔、苏州虎丘塔等。这种装饰不仅美观，而且具有一定的文化意义。然而，在北方地区，虽然也有类似图案的出现，但在辽、金建筑上却未见此式，反映了南北建筑装饰风格的差异。此外，《法式》中的令栱长度规定也与江南建筑实践相符，进一步证明了《法式》与江南建筑的紧密联系。\n\n【问题】\n《营造法式》为什么被认为是一种建筑工程预算定额？\n\n【回答】\n', additional_kwargs={}, re

In [51]:
# ===== 4. 生成最终回答 =====
answer = llm.invoke(final_prompt)

print("=" * 80)
print("Final Answer:\n")
print(answer)


Final Answer:

《营造法式》被认为是一种建筑工程预算定额，是因为其更注重用料的规范性和经济性，而非全面记录雕刻艺术的多样性。 [chunk_id: yingzao_2017_ch1_sec3_0003, toc_path: 第一章 总论,三、《营造法式》的内容取舍, pages: 14-15]


# 对比rag和无rag回答

In [ ]:
def compare_rag_vs_no_rag(
    query: str,
    llm,
    retriever,
    prompt,
    format_docs_fn,
    top_k: int = 5
):
    """
    对同一个 query，对比：
    1. 无 RAG（纯 LLM）
    2. 有 RAG（Retriever + Context + LLM）

    参数说明：
    - llm: LangChain LLM 实例（与你 rag_chain 使用的同一个）
    - retriever: 向量检索器
    - prompt: ChatPromptTemplate（RAG 用）
    - format_docs_fn: 将 docs 转成 context 的函数
    """

    print("=" * 100)
    print("QUERY:")
    print(query)

    # ------------------------------------------------------------------
    # 1. 无 RAG（Baseline）
    # ------------------------------------------------------------------
    print("\n" + "=" * 100)
    print("BASELINE ANSWER (NO RAG):\n")

    baseline_answer = llm.invoke(query)
    print(baseline_answer)

    # ------------------------------------------------------------------
    # 2. 有 RAG：检索阶段
    # ------------------------------------------------------------------
    print("\n" + "=" * 100)
    print("RAG RETRIEVAL STAGE:\n")

    retrieved_docs = retriever.invoke(query)

    for i, doc in enumerate(retrieved_docs[:top_k], start=1):
        print("-" * 80)
        print(f"[Chunk {i}]")
        print("chunk_id:", doc.metadata.get("chunk_id"))
        print("chapter:", doc.metadata.get("chapter"))
        print("pages:", doc.metadata.get("page_start"), "-", doc.metadata.get("page_end"))
        print(doc.page_content[:400])

    # ------------------------------------------------------------------
    # 3. 构造 RAG Context
    # ------------------------------------------------------------------
    print("\n" + "=" * 100)
    print("RAG CONTEXT PASSED TO LLM:\n")

    context_text = format_docs_fn(retrieved_docs[:top_k])
    print(context_text[:2000])

    # ------------------------------------------------------------------
    # 4. 有 RAG：生成阶段
    # ------------------------------------------------------------------
    print("\n" + "=" * 100)
    print("RAG ANSWER:\n")

    rag_prompt = prompt.invoke({
        "context": context_text,
        "question": query
    })

    rag_answer = llm.invoke(rag_prompt)
    print(rag_answer)

    # ------------------------------------------------------------------
    # 5. 返回结构化结果（方便后续评估 / 写论文）
    # ------------------------------------------------------------------
    return {
        "query": query,
        "baseline_answer": baseline_answer,
        "rag_answer": rag_answer,
        "retrieved_chunks": retrieved_docs[:top_k]
    }


In [56]:
query = "江南建筑和营造法式的关系"

result = compare_rag_vs_no_rag(
    query=query,
    llm=llm,
    retriever=retriever,
    prompt=prompt,
    format_docs_fn=format_docs_with_citation,
    top_k=5
)


QUERY:
江南建筑和营造法式的关系

BASELINE ANSWER (NO RAG):

江南建筑和营造法式之间存在着复杂而密切的关系，它们在历史上相互影响、借鉴和发展，共同塑造了中国古代建筑的辉煌成就。要理解它们的关系，需要从以下几个方面进行分析：

**1. 营造法式：源头与基础**

* **起源：** 营造法式是唐朝时期由隋朝开始，在唐朝时期发展成熟的建筑理论体系和规范。它主要由《营造典范》和《营造实用典则》两部典籍构成。
* **核心思想：** 营造法式强调“以形为基，以理为本”，注重建筑的结构、材料、尺寸、比例、装饰等方面，力求达到和谐、稳定、美观的效果。它提出了“三尺为基”、“四尺为梁”、“五尺为柱”等基本尺寸标准，并对建筑的结构、材料、施工工艺等方面进行了详细的规定。
* **影响：** 营造法式对中国古代建筑产生了深远的影响，成为中国古代建筑理论和实践的基础。

**2. 江南建筑：继承与发展**

* **地理环境：** 江南地区地势平坦，水网密布，气候温和，非常适合建筑发展。同时，江南地区经济发达，工匠技术精湛，为建筑的发展提供了良好的条件。
* **继承营造法式：** 江南建筑在继承营造法式的基础上，进行了大量的创新和发展。它吸收了营造法式中的许多理论和技术，并根据江南地区的实际情况进行了调整和完善。
* **特点：** 江南建筑形成了独特的风格，主要特点包括：
    * **园林建筑：** 江南建筑最显著的特点就是园林建筑，如拙政园、留园、香山园等，这些园林以其精巧的设计、优美的景色、丰富的文化内涵而闻名于世。
    * **木结构建筑：** 江南建筑以木结构为主，注重木材的运用和加工，形成了独特的木结构建筑体系。
    * **装饰艺术：** 江南建筑的装饰艺术非常丰富，包括彩绘、雕刻、砖雕、石雕等，这些装饰艺术不仅美化了建筑，也反映了江南地区的文化和审美情趣。
    * **注重细节：** 江南建筑对细节的把握非常精细，如屋顶的坡度、柱子的尺寸、窗户的形状等，都经过精心设计和制作，体现了江南工匠的智慧和技艺。

**3. 相互影响与借鉴**

* **营造法式对江南建筑的影响：** 营造法式为江南建筑提供了理论基础和技术指导，江南建筑在继承营造法式的基础上，进行了大量的创新和发展。
* **江南建筑对营造法式的补充与完善：